# Experiment

> Scripts to perform the experiments

In [1]:
#| default_exp experiment

In [2]:
#| export
#| hide
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Optional, List, Union, Tuple
import torch
import itertools
import nbformat
import papermill as pm
import torch.nn as nn
import re
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed

In [3]:
#| hide
from fastcore.test import test_eq

## JSON Management

### Setup

In [4]:
#| export
def setup_new_experiment(params: Dict[str, Any],              # Dictionary of parameters for the new experiment.
                         experiments_folder: str,             # Path to the folder containing all experiments.
                         json_file: Optional[str] = None      # Optional path to the JSON file tracking experiment parameters.
                        ) -> str:                             # The path to the newly created experiment folder.
    """
    Sets up a new experiment by creating a new folder and updating the JSON file with experiment parameters.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        os.makedirs(experiments_folder)

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    # Load existing experiments from the JSON file if it exists
    if os.path.isfile(json_file):
        with open(json_file, mode='r') as file:
            experiments = json.load(file)
    else:
        experiments = []

    # Check if the parameters already exist in the JSON file
    for experiment in experiments:
        if all(experiment['parameters'].get(key) == value for key, value in params.items()):
            candidate_folder = os.path.join(experiments_folder, f"experiment_{experiment['id']}")
            if os.path.exists(candidate_folder):
                print(f'Parameters already exist for experiment: {candidate_folder}')
                return candidate_folder

    # Determine the next experiment number
    next_experiment_number = max((experiment['id'] for experiment in experiments), default=0) + 1

    # Create a new folder for the next experiment
    new_experiment_folder = os.path.join(experiments_folder, f'experiment_{next_experiment_number}')
    os.makedirs(new_experiment_folder, exist_ok=True)

    # Add the new experiment to the list and save to JSON file
    new_experiment = {
        'id': next_experiment_number,
        'parameters': params
    }
    experiments.append(new_experiment)
    with open(json_file, mode='w') as file:
        json.dump(experiments, file, indent=4)

    print(f'New experiment setup complete: {new_experiment_folder}')
    print(f'Parameters saved to {json_file}.')

    return new_experiment_folder

In [5]:
#| export
def create_experiments_json(parameter_sets, output_file='experiments.json'):
    """
    Create an experiments.json file from given parameter sets.
    
    Args:
    parameter_sets (list): List of dictionaries containing parameters for each experiment.
    output_file (str): Name of the output JSON file. Defaults to 'experiments.json'.
    
    Returns:
    None
    """
    experiments = []
    
    for i, params in enumerate(parameter_sets, start=1):
        experiment = {
            "id": i,
            "parameters": params
        }
        experiments.append(experiment)
    
    with open(output_file, 'w') as f:
        json.dump(experiments, f, indent=4)
    
    print(f"Experiments JSON file created: {output_file}")

### Add Metrics

In [6]:
#| export
def convert_numpy_types(obj):
    """
    Recursively convert numpy types and tensors to native Python types for JSON serialization.
    """
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, torch.Tensor):
        return obj.item() if obj.numel() == 1 else obj.tolist()
    else:
        return obj

In [7]:
#| export
def add_experiment_metrics(experiments_folder: str,                    # Path to the folder containing all experiments.
                           params: Optional[Dict[str, Any]] = None,    # Optional dictionary of parameters identifying the experiment.
                           experiment_id: Optional[int] = None,        # Optional ID to identify the experiment.
                           metrics: Optional[Dict[str, Any]] = None,   # Optional dictionary of metrics to be added to the experiment.
                           json_file: Optional[str] = None             # Optional path to the JSON file tracking experiment parameters and metrics.
                          ) -> None:
    """
    Adds metrics to an existing experiment in the JSON file based on the given parameters or ID.
    """
    if not os.path.exists(experiments_folder):
        raise FileNotFoundError(f"The experiments folder '{experiments_folder}' does not exist.")

    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    if not os.path.isfile(json_file):
        raise FileNotFoundError(f"The JSON file '{json_file}' does not exist.")

    if params is None and experiment_id is None:
        raise ValueError("Either 'params' or 'experiment_id' must be provided to identify the experiment.")

    if metrics is None:
        metrics = {}

    with open(json_file, mode='r') as file:
        try:
            experiments = json.load(file)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error reading JSON file: {e}")

    found_experiment = False

    for experiment in experiments:
        if (experiment_id is not None and experiment['id'] == experiment_id) or \
           (params is not None and all(experiment['parameters'].get(key) == value for key, value in params.items())):
            experiment.update(convert_numpy_types(metrics))
            found_experiment = True
            break

    if not found_experiment:
        if experiment_id is not None:
            raise ValueError(f"Experiment with the specified ID {experiment_id} does not exist.")
        else:
            raise ValueError("Experiment with the specified parameters does not exist.")

    # Convert the entire experiments list to ensure all nested objects are serializable
    serializable_experiments = convert_numpy_types(experiments)

    with open(json_file, mode='w') as file:
        json.dump(serializable_experiments, file, indent=4)

    if experiment_id is not None:
        print(f'Metrics added to experiment with ID {experiment_id} in {json_file}.')
    else:
        experiment_id = experiment['id']
        print(f'Metrics added to experiment with ID {experiment_id} in {json_file}.')

### Get Metrics

In [8]:
#| export
def get_experiment_parameters(experiments_folder: str,                    # Path to the folder containing all experiments.
                              experiment_id: int,                         # ID to identify the experiment.
                              json_file: Optional[str] = None             # Optional path to the JSON file tracking experiment parameters and metrics.
                             ) -> Dict[str, Any]:
    """
    Retrieves the parameters of an experiment from the JSON file based on the given ID.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        raise FileNotFoundError(f"The experiments folder '{experiments_folder}' does not exist.")

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    if not os.path.isfile(json_file):
        raise FileNotFoundError(f"The JSON file '{json_file}' does not exist.")

    # Load existing experiments from the JSON file
    with open(json_file, mode='r') as file:
        experiments = json.load(file)

    # Find the matching experiment and return its parameters
    for experiment in experiments:
        if experiment['id'] == experiment_id:
            return experiment.get('parameters', {})

    # If the experiment is not found, raise an error
    raise ValueError(f"Experiment with the specified ID {experiment_id} does not exist.")

### Get Data

In [9]:
#| export
def get_experiment_data(experiments_folder: str,  # Path to the folder containing all experiments
                       experiment_id: int,  # ID to identify the experiment
                       json_file: Optional[str] = None,  # Optional path to the JSON file tracking experiment data
                       ) -> Dict[str, Any]:
    """
    Retrieves all data for an experiment from the JSON file based on the given ID.
    """
    # Ensure the experiments folder exists
    if not os.path.exists(experiments_folder):
        raise FileNotFoundError(f"The experiments folder '{experiments_folder}' does not exist.")

    # Default JSON file to 'experiments.json' in the experiments_folder if not provided
    if json_file is None:
        json_file = os.path.join(experiments_folder, 'experiments.json')

    if not os.path.isfile(json_file):
        raise FileNotFoundError(f"The JSON file '{json_file}' does not exist.")

    # Load existing experiments from the JSON file
    with open(json_file, mode='r') as file:
        experiments = json.load(file)

    # Find the matching experiment and return all its data
    for experiment in experiments:
        if experiment['id'] == int(experiment_id):
            return experiment

    # If the experiment is not found, raise an error
    raise ValueError(f"Experiment with the specified ID {experiment_id} does not exist.")

In [10]:
#| export
def read_json_to_dataframe(json_path: str,  # Path to the JSON file containing experiment results
                          ) -> pd.DataFrame:
    """
    Reads a JSON file containing experiment results and returns a DataFrame.
    """
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    # Extract relevant information
    records = []
    for item in data:
        # Assuming each item is a dictionary with an 'id' field and other details
        record = {'id_experiment': item['id']}
        record.update(item)
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df

## Data Management

### Get Paths

In [11]:
#| export
def generate_image_paths(folder_prefix: str,  # Base folder path prefix for each experiment
                        unique_ids: list[int],  # List of experiment IDs
                        file_suffix: str,  # Suffix to append to the generated filenames
                        ) -> list[str]:
    """
    Generates a list of image file paths based on experiment IDs and folder structure.
    """
    file_paths = []
    for unique_id in unique_ids:
        file_name = f"exp{unique_id}{file_suffix}"
        file_path = f"{folder_prefix}{unique_id}/images/{file_name}"
        file_paths.append(file_path)
    return file_paths

### Get Orbits

In [12]:
#| export
def concatenate_orbits_from_experiment_folder(experiments_folder: str,  # Root folder containing experiment subfolders
                                            seq_len: int,  # Expected sequence length of orbit data
                                            file_suffix: str = '_generated_orbits',  # Suffix for orbit data files
                                            ) -> np.ndarray:
    """
    Concatenates orbit data from multiple experiment folders into a single array.
    """
    arrays = []
    
    for folder in os.listdir(experiments_folder):
        if folder.startswith('experiment_') and os.path.isdir(os.path.join(experiments_folder, folder)):
            # Extract the experiment number using regex
            match = re.search(r'experiment_(\d+)', folder)
            if match:
                experiment_id = match.group(1)
                generated_data_path = os.path.join(experiments_folder, folder, f'exp{experiment_id}{file_suffix}.npy')
                
                if os.path.isfile(generated_data_path):
                    generated_orbit = np.load(generated_data_path)
                    
                    if generated_orbit.shape[-1] == seq_len:
                        arrays.append(generated_orbit)
    
    if arrays:
        return np.concatenate(arrays, axis=0)
    else:
        return np.array([])

### Get Tables

In [13]:
#| export
def concatenate_csvs_from_experiment_folder(experiments_folder: str,  # Root folder containing experiment subfolders
                                          file_suffix: str,  # Suffix for CSV files to concatenate
                                          ) -> pd.DataFrame:
    """
    Concatenates CSV files from multiple experiment folders into a single DataFrame.
    """
    dataframes = []
    
    for folder in os.listdir(experiments_folder):
        if folder.startswith('experiment_') and os.path.isdir(os.path.join(experiments_folder, folder)):
            # Extract the experiment number using regex
            match = re.search(r'experiment_(\d+)', folder)
            if match:
                experiment_id = match.group(1)
                csv_file_path = os.path.join(experiments_folder, folder, f'exp{experiment_id}_{file_suffix}.csv')
                
                if os.path.isfile(csv_file_path):
                    # Load the CSV file into a DataFrame
                    df = pd.read_csv(csv_file_path)
                    
                    # Add a column to identify the experiment
                    df['experiment_id'] = experiment_id
                    
                    dataframes.append(df)
    
    # Concatenate all DataFrames along rows
    if dataframes:
        return pd.concat(dataframes, axis=0, ignore_index=True)
    else:
        return pd.DataFrame()

### Get Orbit and Table

In [14]:
#| export
def concatenate_and_check_orbits_from_experiment_folder(
    experiments_folder: str,  # Root folder containing experiment subfolders
    csv_file_name: str = '_refined_orbits_df.csv',  # Suffix for CSV files containing refined orbits
    np_file_name: str = '_refined_orbits',  # Suffix for numpy files containing generated orbits
    ) -> Tuple[np.ndarray, pd.DataFrame]:
    """
    Concatenates orbit data from multiple experiment folders and checks for consistency between generated and refined orbits.
    """
    arrays = []
    all_refined_orbit_dfs = []
    
    for folder in os.listdir(experiments_folder):
        folder_path = os.path.join(experiments_folder, folder)  # Construct the folder path
        
        if folder.startswith('experiment_') and os.path.isdir(folder_path):
            # Extract the experiment number using regex
            match = re.search(r'experiment_(\d+)', folder)
            if match:
                experiment_id = match.group(1)
                generated_data_path = os.path.join(folder_path, f'exp{experiment_id}{np_file_name}.npy')
                refined_orbit_path = os.path.join(folder_path, f'exp{experiment_id}{csv_file_name}')  # Path to refined orbits
                
                if os.path.isfile(generated_data_path):
                    generated_orbit = np.load(generated_data_path)
                    
                    # Load the refined orbits DataFrame
                    if os.path.isfile(refined_orbit_path):
                        refined_orbit_df = pd.read_csv(refined_orbit_path)
                        
                        # Add a column to identify the experiment
                        refined_orbit_df['experiment_id'] = experiment_id
                        
                        # Check if the number of orbits matches the length of refined_orbit_df
                        if generated_orbit.shape[0] != len(refined_orbit_df):
                            print(f"Mismatch for experiment {experiment_id}: generated_orbit count = {generated_orbit.shape[0]}, refined_orbit_df length = {len(refined_orbit_df)}")
                            continue  # Skip to the next folder if there is a mismatch
                        
                        all_refined_orbit_dfs.append(refined_orbit_df)
                    
                    arrays.append(generated_orbit)
    
    if arrays:
        concatenated_orbit = np.concatenate(arrays, axis=0)
        return concatenated_orbit, pd.concat(all_refined_orbit_dfs, ignore_index=True)
    else:
        return np.array([]), pd.DataFrame()

## Parameters

In [15]:
#| export
def generate_parameter_sets(params: Dict[str, Union[Any, List[Any]]],  # Dictionary of parameter names and their values/value lists
                          model_specific_params: Dict[str, Dict],  # Dictionary mapping model names to their specific parameters
                          ) -> List[Dict]:
    """
    Generates all possible parameter combinations from the given parameter sets and merges them with model-specific parameters.
    """
    keys, values = zip(*params.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*[
        value if isinstance(value, list) else [value] for value in values
    ])]
    
    final_combinations = []
    for combo in combinations:
        model_name = combo['model_name']
        if model_name in model_specific_params:
            model_kwargs = model_specific_params[model_name].copy()
            combo['model_kwargs'] = model_kwargs
            combo['model_kwargs']['beta'] = combo.pop('beta')
            final_combinations.append(combo)
    
    return final_combinations


## Experiment Execution

In [16]:
#| export
def execute_parameter_notebook(notebook_to_execute: str,  # Path to the notebook file to execute
                             output_dir: str,  # Directory to save output notebook
                             i: int,  # Execution index
                             params: Dict,  # Parameters to pass to the notebook
                             extra_parameters: Optional[Dict] = None,  # Additional parameters to merge with params
                             checkpoint_file: Optional[str] = None,  # Path to checkpoint file
                             ) -> Optional[int]:
    """
    Executes a Jupyter notebook with given parameters and saves the output.
    Returns the execution index if successful, None if failed.
    """
    try:
        # Mark as started
        with open(checkpoint_file, 'r+') as f:
            checkpoint = json.load(f)
            if i not in checkpoint['started']:
                checkpoint['started'].append(i)
                f.seek(0)
                json.dump(checkpoint, f)
                f.truncate()
        
        logging.info(f"Starting execution {i}")

        # Generate output filenames
        base_name = os.path.splitext(os.path.basename(notebook_to_execute))[0]
        output_notebook = os.path.join(output_dir, f"{base_name}_execution_{i}.ipynb")

        # Read the notebook
        with open(notebook_to_execute, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)

        # Merge params and extra_parameters into a single dictionary
        if extra_parameters is not None:
            params.update(extra_parameters)

        nb = pm.execute_notebook(
            nb,
            output_notebook,
            parameters=params,
            kernel_name='pytorch',
            timeout=100000,
            log_output=True
        )
        
        logging.info(f"Completed execution {i}")
        return i

    except Exception as e:
        logging.error(f"Error in execution {i}: {str(e)}")
        logging.error(f"Parameters used: {params}")
        import traceback
        logging.error(f"Traceback: {traceback.format_exc()}")
        return None

In [17]:
#| export
def paralelize_notebook_experiment(parameter_sets: List[Dict],  # List of parameter dictionaries to execute
                                 notebook_to_execute: str,  # Path to the notebook file to execute
                                 output_dir: str,  # Directory to save execution outputs
                                 checkpoint_file: str,  # Path to checkpoint file tracking execution progress
                                 max_workers: int = 3,  # Maximum number of parallel workers
                                 extra_parameters: Optional[Dict] = None,  # Additional parameters to pass to each execution
                                 ) -> None:
    """
    Executes a Jupyter notebook multiple times in parallel with different parameter sets, tracking progress with checkpoints.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
    else:
        checkpoint = {'completed': [], 'started': []}
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)

    # Ensure checkpoint is a dictionary with 'completed' and 'started' keys
    if not isinstance(checkpoint, dict) or 'completed' not in checkpoint or 'started' not in checkpoint:
        checkpoint = {'completed': [], 'started': []}
        with open(checkpoint_file, 'w') as f:
            json.dump(checkpoint, f)
    
    # Filter out already completed executions
    remaining_executions = [i for i in range(1, len(parameter_sets) + 1) if i not in checkpoint['completed']]
    
    logging.info(f"Starting execution. {len(remaining_executions)} executions remaining.")
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i in remaining_executions:
            future = executor.submit(
                execute_parameter_notebook,
                notebook_to_execute=notebook_to_execute,
                output_dir=output_dir,
                i=i,
                params=parameter_sets[i-1],
                extra_parameters=extra_parameters,
                checkpoint_file=checkpoint_file
            )
            futures.append(future)
        
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                logging.info(f"Execution {result} completed successfully.")
                # Update checkpoint
                with open(checkpoint_file, 'r+') as f:
                    checkpoint = json.load(f)
                    checkpoint['completed'].append(result)
                    f.seek(0)
                    json.dump(checkpoint, f)
                    f.truncate()
            else:
                logging.warning("An execution failed.")
    
    logging.info("All executions completed.")

## Functions

In [18]:
#| export
data_path = r"/orbit-generation/data/orbits_fix_1500/EM_N_fix_1500.h5"
experiments_folder = "../experiments"

In [19]:
#| export
def generate_file_paths(experiment_id: Union[int, str],  # Unique ID of the experiment
                       images_folder: str,  # Folder path where image files are stored
                       experiment_folder: str,  # Folder path where experiment-related files are stored
                       ) -> Dict[str, str]:
    """
    Generate a dictionary of file paths for an experiment.
    """
    paths = {
        # Images - Orbits
        'static_all_orbit_path': os.path.join(images_folder, f'exp{experiment_id}_generated_orbits.png'),
        'dynamic_orbits_path': os.path.join(images_folder, f'exp{experiment_id}_generated_orbits.html'),
        'refined_orbits_path': os.path.join(images_folder, f'exp{experiment_id}_refined_orbits.png'),
        'dynamical_refined_orbits_path': os.path.join(images_folder, f'exp{experiment_id}_refined_orbits.html'),
        'generated_orbits_that_converged_path': os.path.join(images_folder, f'exp{experiment_id}_generated_orbits_that_converged.png'),
        'generated_orbits_that_did_not_converged_path': os.path.join(images_folder, f'exp{experiment_id}_generated_orbits_that_not_converged.png'),

        # Images - Latent Spaces
        'latent_space_path': os.path.join(images_folder, f'exp{experiment_id}_latent_space'),
        'full_latent_space_path': os.path.join(images_folder, f'exp{experiment_id}_full_latent_space'),
        'discarded_latent_space_path': os.path.join(images_folder, f'exp{experiment_id}_discarded_latent_space'),
        'combined_latent_space_path': os.path.join(images_folder, f'exp{experiment_id}_combined_latent_space'),
        'combined_latent_space_arrows_path': os.path.join(images_folder, f'exp{experiment_id}_combined_latent_space_arrows'),
        'family_centroids_plot_path': os.path.join(images_folder, f'exp{experiment_id}_family_centroids'),
        'full_family_centroids_plot_path': os.path.join(images_folder, f'exp{experiment_id}_full_family_centroids'),

        # Images - Feature Spaces
        'features_plot_path': os.path.join(images_folder, f'exp{experiment_id}_features'),
        'family_feature_centroids_plot_path': os.path.join(images_folder, f'exp{experiment_id}_family_feature_centroids'),

        # Other Images
        'model_losses_path': os.path.join(images_folder, f'exp{experiment_id}_model_losses.png'),
        'histogram_comparison_path': os.path.join(images_folder, f'exp{experiment_id}_histogram_comparison.png'),
        'full_histogram_comparison_path': os.path.join(images_folder, f'exp{experiment_id}_full_histogram_comparison.png'),

        # Model
        'model_save_path': os.path.join(experiment_folder, f'exp{experiment_id}_model.pth'),

        # Orbits Data
        'generated_data_path': os.path.join(experiment_folder, f'exp{experiment_id}_generated_orbits.npy'),
        'refined_data_path': os.path.join(experiment_folder, f'exp{experiment_id}_refined_orbits.npy'),

        # Latent Representations
        'latent_representations_path': os.path.join(experiment_folder, f'exp{experiment_id}_latent_representations.npy'),
        'family_centroids_path': os.path.join(experiment_folder, f'exp{experiment_id}_family_centroids.npy'),

        # Features Data
        'generation_df_path': os.path.join(experiment_folder, f'exp{experiment_id}_generation_df.csv'),
        'refined_orbits_df_path': os.path.join(experiment_folder, f'exp{experiment_id}_refined_orbits_df.csv')
    }
    return paths

In [20]:
#| export
from orbit_generation.dataset import get_first_period_dataset, get_orbit_classes
from orbit_generation.data import TSFeatureWiseScaler, discard_random_labels
import torch

In [21]:
#| export
def prepare_experiment_data(params: Dict[str, Any],  # Dictionary containing experiment parameters
                          experiments_folder: str,  # Folder where experiments are stored
                          data_path: str,  # Path to the dataset file
                          want_to_discover: bool,  # Flag indicating whether to discover new families or use existing ones
                          ) -> Tuple[torch.Tensor, pd.DataFrame, List[str], List[str], np.ndarray, Dict[str, str], int]:
    """
    Prepare the experiment data based on the provided parameters and configurations.
    """
    # Step 1: Setup experiment folders
    experiment_folder = setup_new_experiment(params, experiments_folder)
    images_folder = os.path.join(experiment_folder, 'images')
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    
    experiment_id = int(os.path.basename(experiment_folder).split('_')[1])

    # Step 2: Generate file paths
    file_paths = generate_file_paths(experiment_id, images_folder, experiment_folder)

    # Step 3: Load full dataset
    full_data, full_orbit_df, full_labels, system_dict = get_first_period_dataset(
        file_path=data_path,
        segment_length=params['seq_len']
    )

    # Adjust data shape if feature_dim is 6
    if params['feature_dim'] == 6:
        full_data = full_data[:, 1:, :]  # Remove the first feature dimension

    # Map orbit IDs to classes
    full_orbits_id_classes = [full_orbit_df.at[index, 'id_class'] for index in full_labels]

    # Step 4: Handle discovery or reuse of discarded families
    if want_to_discover:
        discarded_family_ids, data, orbits_id_classes = discard_random_labels(
            full_data,
            np.array(full_orbits_id_classes),
            int(params['families_to_discard'])
        )
        discarded_families = get_orbit_classes(discarded_family_ids)[0]
        orbit_df = full_orbit_df[~full_orbit_df['id_class'].isin(discarded_family_ids)]

        # Log metrics for the experiment
        add_experiment_metrics(
            experiments_folder,
            experiment_id=experiment_id,
            metrics={
                'discarded_family_ids': discarded_family_ids,
                'discarded_families': discarded_families
            }
        )
    else:
        # Retrieve previously discarded families for this experiment
        experiment_data = get_experiment_data(
            experiments_folder=experiments_folder,
            experiment_id=experiment_id
        )
        discarded_family_ids = experiment_data.get('discarded_family_ids', {})

        discarded_family_ids, data, orbits_id_classes = discard_random_labels(
            full_data,
            np.array(full_orbits_id_classes),
            discarded_family_ids
        )
        discarded_families = get_orbit_classes(discarded_family_ids)[0]
        orbit_df = full_orbit_df[~full_orbit_df['id_class'].isin(discarded_family_ids)]

    # Step 5: Get family labels for remaining orbits
    family_labels = get_orbit_classes(orbits_id_classes)[0]

    # Step 6: Extract features and scale data
    feature_names = ['jacobi', 'period', 'stability']
    features = orbit_df[feature_names].to_numpy()

    scaler = TSFeatureWiseScaler()
    scaled_data = scaler.fit_transform(torch.tensor(data, dtype=torch.float32))

    # Return processed data and metadata
    return scaled_data, orbit_df, family_labels, discarded_families, features, file_paths, experiment_id


In [22]:
#| export
from pytorch_lightning import Trainer
from orbit_generation.model_factory import get_model
from orbit_generation.data import create_dataloaders
from orbit_generation.architectures import VAELossHistory

In [ ]:
#| export
def prepare_and_train_model(params: Dict[str, Any],  # Dictionary containing all experiment parameters
                           scaled_data: torch.Tensor,  # Scaled data tensor for training/validation
                           experiments_folder: str,  # Folder where experiments are stored
                           experiment_id: int,  # Unique ID of current experiment
                           file_paths: Dict[str, str],  # Dictionary of paths for model/metrics files
                           want_to_train: bool,  # Whether to train model or load pre-trained
                           ) -> nn.Module:
    """
    Prepare the model and either train it or load a pre-trained version based on the provided parameters.
    """
    # Step 1: Initialize the model
    model = get_model(params)

    # Step 2: Handle training
    if want_to_train:
        # Create data loaders
        train_loader, val_loader = create_dataloaders(
            scaled_data,
            val_split=params.get('val_split', 0.1),
            batch_size=params.get('batch_size', 32)
        )

        # Initialize loss history callback
        loss_history = VAELossHistory()

        # Initialize Trainer
        trainer = Trainer(
            max_epochs=params.get('epochs', 10),
            log_every_n_steps=10,
            devices="auto",
            accelerator="auto",
            enable_progress_bar=True,
            enable_model_summary=True,
            callbacks=[loss_history]
        )

        # Train the model
        trainer.fit(model, train_loader, val_loader)

        # Log metrics
        for metric_name, metric_value in trainer.callback_metrics.items():
            print(f"{metric_name}: {metric_value}")

        # Save metrics to experiment folder
        add_experiment_metrics(
            experiments_folder,
            experiment_id=experiment_id,
            metrics=trainer.callback_metrics
        )

        # Save model state to file
        torch.save(model.state_dict(), file_paths['model_save_path'])

        # Plot and save loss history
        loss_history.plot_all_losses(save_path=file_paths['model_losses_path'])

    else:
        # Load pre-trained model state from file
        model.load_state_dict(torch.load(file_paths['model_save_path'], weights_only=True))

    return model

In [24]:
#| hide
import nbdev; nbdev.nbdev_export()